In [1]:
import sqlite3
from typing import List, Optional


class WikiMapper:
    """Uses a precomputed database created by `create_wikipedia_wikidata_mapping_db`."""

    def __init__(self, path_to_db: str):
        self._path_to_db = path_to_db
        self.conn = sqlite3.connect(self._path_to_db)

    def title_to_id(self, page_title: str) -> Optional[str]:
        """Given a Wikipedia page title, returns the corresponding Wikidata ID.
        The page title is the last part of a Wikipedia url **unescaped** and spaces
        replaced by underscores , e.g. for `https://en.wikipedia.org/wiki/Fermat%27s_Last_Theorem`,
        the title would be `Fermat's_Last_Theorem`.
        Args:
            page_title: The page title of the Wikipedia entry, e.g. `Manatee`.
        Returns:
            Optional[str]: If a mapping could be found for `wiki_page_title`, then return
                           it, else return `None`.
        """

        c = self.conn.execute("SELECT wikidata_id FROM mapping WHERE wikipedia_title=?", (page_title,))
        result = c.fetchone()

        if result is not None and result[0] is not None:
            return result[0]
        else:
            return None

    def url_to_id(self, wiki_url: str) -> Optional[str]:
        """Given an URL to a Wikipedia page, returns the corresponding Wikidata ID.
        This is just a convenience function. It is not checked whether the index and
        URL are from the same dump.
        Args:
            wiki_url: The URL to a Wikipedia entry.
        Returns:
            Optional[str]: If a mapping could be found for `wiki_url`, then return
                           it, else return `None`.
        """

        title = wiki_url.rsplit("/", 1)[-1]
        return self.title_to_id(title)

    def id_to_titles(self, wikidata_id: str) -> List[str]:
        """Given a Wikidata ID, return a list of corresponding pages that are linked to it.
        Due to redirects, the mapping from Wikidata ID to Wikipedia title is not unique.
        Args:
            wikidata_id (str): The Wikidata ID to map, e.g. `Q42797`.
        Returns:
            List[str]: A list of Wikipedia pages that are linked to this Wikidata ID.
        """

        c = self.conn.execute(
            "SELECT DISTINCT wikipedia_title FROM mapping WHERE wikidata_id =?", (wikidata_id,)
        )
        results = c.fetchall()

        return [e[0] for e in results]

# CHICAGO

In [3]:

import os

map = WikiMapper('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/index_enwiki-20190420.db')

definitions = '/Users/lucyhorowitz/Library/Mobile Documents/iCloud~md~obsidian/Documents/MathRepository/notes/definitions'

wiki = open('chitest.txt', 'a')
towrite = []
for filepath in os.listdir(definitions):
    defn = os.path.join(definitions, filepath)
    defn_text = open(defn).read()
    
    ids = [url[30:] for url in defn_text.split() if url.__contains__('wikidata')]
    if ids:
        for thing in ids:
            towrite.append(thing + ': ' + filepath[:-3] + '\n')
        #wiki.write('\n')

for thing in towrite:       
    wiki.write(thing)


# PROOFNET

In [9]:
# proofnet is small, so no cutoff

map = WikiMapper('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/wikidata/index_enwiki-20190420.db')

wikicats = ['_(mathematics)', '_(linear_algebra)', '_(algebraic_geometry)', '_(algebraic_topology)', '_(calculus)', '_(category_theory)',
             '_(commutative_algebra)', '_(field_theory)', '_(game_theory)', '_(topology)', '_(differential_geometry)', '_(graph_theory)', 
             '_(group_theory)', '_(invariant_theory)', '_(module_theory)', '_(order_theory)', '_(probability)', '_(statistics)', '_(ring_theory)',
             '_(representation_theory)', '_(set_theory)', '_(string_theory)', '_(symplectic geometry)', '_(tensor_theory)']

wiki = open('pn-plus.txt', 'a')
#wiki.write('ADJ-NOUN PAIRS:\n')
for line in open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/proofnet/pn-ans.txt', 'r').readlines():
    line = line.replace(' - ', '-')
    line = line.replace('$', '')
    line = line.replace(' ', '_')
    line = line[0].upper() + line[1:line.index(':')]
    linesplus = [line + cat for cat in wikicats]
    exclude = False
    for thing in linesplus:
        if map.title_to_id(thing):
            exclude = True
    if not exclude:
        linesplus.append(line)
    for thing in linesplus:
        if map.title_to_id(thing):
            wiki.write(thing + ': ' + map.title_to_id(thing)  + '\n')

#wiki.write('\nCOMPOUNDS:\n')
for line in open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/proofnet/pn-compounds.txt', 'r').readlines():
    line = line.replace(' - ', '-')
    line = line.replace('$', '')
    line = line.replace(' ', '_')
    line = line[0].upper() + line[1:line.index(':')]
    linesplus = [line + cat for cat in wikicats]
    exclude = False
    for thing in linesplus:
        if map.title_to_id(thing):
            exclude = True
    if not exclude:
        linesplus.append(line)
    for thing in linesplus:
        if map.title_to_id(thing):
            wiki.write(thing + ': ' + map.title_to_id(thing)  + '\n')

#wiki.write('\nNOUNS:\n')
for line in open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/proofnet/pn-nouns.txt', 'r').readlines():
    line = line.replace(' - ', '-')
    line = line.replace('$', '')
    line = line.replace(' ', '_')
    line = line[0].upper() + line[1:line.index(':')]
    linesplus = [line + cat for cat in wikicats]
    exclude = False
    for thing in linesplus:
        if map.title_to_id(thing):
            exclude = True
    if not exclude:
        linesplus.append(line)
    for thing in linesplus:
        if map.title_to_id(thing):
            wiki.write(thing + ': ' + map.title_to_id(thing)  + '\n')
#wiki.write('\nNOUNS:\n')
#for line in open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/tac /hyph-propns.txt', 'r').readlines():
    #line = line.replace(' - ', '-')
    #line = line.replace('$', '')
    #line = line.replace(' ', '_')
    #line = line[0].upper() + line[1:line.index(':')]
    #if map.title_to_id(line):
        #wiki.write(line + ': ' + map.title_to_id(line)  + '\n')


# FRANCE

In [3]:
map = WikiMapper('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/wikidata/index_enwiki-20190420.db')

wikicats = ['_(mathematics)', '_(linear_algebra)', '_(algebraic_geometry)', '_(algebraic_topology)', '_(calculus)', '_(category_theory)',
             '_(commutative_algebra)', '_(field_theory)', '_(game_theory)', '_(topology)', '_(differential_geometry)', '_(graph_theory)', 
             '_(group_theory)', '_(invariant_theory)', '_(module_theory)', '_(order_theory)', '_(probability)', '_(ring_theory)',
             '_(representation_theory)', '_(set_theory)', '_(string_theory)', '_(symplectic geometry)', '_(tensor_theory)']

wiki = open('france-4.txt', 'a')
#wiki.write('\nIN LEAN:\n')
for text in open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/france/france.txt', 'r').readlines():
    line = text.split(': ')[1].strip()
    line = line.replace(' - ', '-')
    line = line.replace('$', '')
    line = line.replace(' ', '_')
    line = line[0].upper() + line[1:].strip()
    linesplus = [line + cat for cat in wikicats]
    exclude = False
    for thing in linesplus:
        if map.title_to_id(thing):
            exclude = True
    if not exclude:
        linesplus.append(line)
    for thing in linesplus:
        if map.title_to_id(thing):
            wiki.write(thing.replace('_', " ") + "](" + text.split(': ')[0].strip() + '): ' + map.title_to_id(thing) +'\n')

#wiki.write('\nNOT IN LEAN:\n')
for line in open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/france/france-no-lean.txt', 'r').readlines():
    line = line.replace(' - ', '-')
    line = line.replace('$', '')
    line = line.replace(' ', '_')
    line = line[0].upper() + line[1:].strip()
    linesplus = [line + cat for cat in wikicats]
    exclude = False
    for thing in linesplus:
        if map.title_to_id(thing):
            exclude = True
    if not exclude:
        linesplus.append(line)
    for thing in linesplus:
        if map.title_to_id(thing):
            wiki.write(thing.replace('_', ' ') + ': ' + map.title_to_id(thing)  + '\n')
            print('ehllo')

#wiki.write('\nNOUNS:\n')
#for line in open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/tac /hyph-propns.txt', 'r').readlines():
    #line = line.replace(' - ', '-')
    #line = line.replace('$', '')
    #line = line.replace(' ', '_')
    #line = line[0].upper() + line[1:line.index(':')]
    #if map.title_to_id(line):
        #wiki.write(line + ': ' + map.title_to_id(line)  + '\n')

ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo
ehllo


# TAC

In [36]:
#tac cutoff is n = 5

map = WikiMapper('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/wikidata/index_enwiki-20190420.db')

wikicats = ['_(mathematics)', '_(linear_algebra)', '_(algebraic_geometry)', '_(algebraic_topology)', '_(calculus)', '_(category_theory)',
             '_(commutative_algebra)', '_(field_theory)', '_(game_theory)', '_(topology)', '_(differential_geometry)', '_(graph_theory)', 
             '_(group_theory)', '_(invariant_theory)', '_(module_theory)', '_(order_theory)', '_(probability)', '_(statistics)', '_(ring_theory)',
             '_(representation_theory)', '_(set_theory)', '_(string_theory)', '_(symplectic geometry)', '_(tensor_theory)']

wiki = open('tac-plus.txt', 'a')
#wiki.write('ADJ-NOUN PAIRS:\n')
for line in open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/tac/hyph-an-counts.txt', 'r').readlines():
    if int(line[line.index(':') + 2]) >= 5:
        line = line.replace(' - ', '-')
        line = line.replace('$', '')
        line = line.replace(' ', '_')
        line = line[0].upper() + line[1:line.index(':')]
        linesplus = [line + cat for cat in wikicats]
        exclude = False
        for thing in linesplus:
            if map.title_to_id(thing):
                exclude = True
        if not exclude:
            linesplus.append(line)
        for thing in linesplus:
            if map.title_to_id(thing):
                wiki.write(thing + ': ' + map.title_to_id(thing)  + '\n')
                #print(thing)

#wiki.write('\nCOMPOUNDS:\n')
for line in open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/tac/hyph-compounds.txt', 'r').readlines():
    if int(line[line.index(':') + 2]) >= 5:
        line = line.replace(' - ', '-')
        line = line.replace('$', '')
        line = line.replace(' ', '_')
        line = line[0].upper() + line[1:line.index(':')]
        linesplus = [line + cat for cat in wikicats]
        exclude = False
        for thing in linesplus:
            if map.title_to_id(thing):
                exclude = True
        if not exclude:
            linesplus.append(line)
        for thing in linesplus:
            if map.title_to_id(thing):
                wiki.write(thing + ': ' + map.title_to_id(thing)  + '\n')
                #print(thing)

#wiki.write('\nNOUNS:\n')
for line in open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/tac/hyph-nouns.txt', 'r').readlines():
   if int(line[line.index(':') + 2]) >= 5:
        line = line.replace(' - ', '-')
        line = line.replace('$', '')
        line = line.replace(' ', '_')
        line = line[0].upper() + line[1:line.index(':')]
        linesplus = [line + cat for cat in wikicats]
        exclude = False
        for thing in linesplus:
            if map.title_to_id(thing):
                exclude = True
        if not exclude:
            linesplus.append(line)
        for thing in linesplus:
            if map.title_to_id(thing):
                wiki.write(thing + ': ' + map.title_to_id(thing)  + '\n')
                #print(thing)

#wiki.write('\nNOUNS:\n')
#for line in open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/tac /hyph-propns.txt', 'r').readlines():
    #line = line.replace(' - ', '-')
    #line = line.replace('$', '')
    #line = line.replace(' ', '_')
    #line = line[0].upper() + line[1:line.index(':')]
    #if map.title_to_id(line):
        #wiki.write(line + ': ' + map.title_to_id(line)  + '\n')

In [8]:
#make nlab terms into one big list
termlist = []
for line in open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/nlab/nlab-ans.txt', 'r').readlines():
    termlist.append(line[:line.index(':')])
for line in open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/nlab/nlab-compounds.txt', 'r').readlines():
    termlist.append(line[:line.index(':')])
for line in open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/nlab/nlab-nouns.txt', 'r').readlines():
    termlist.append(line[:line.index(':')])
#for line in open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/nlab/nlab-propns.txt', 'r').readlines():
    #termlist.append(line[:line.index(':')])

termlist = sorted(set(termlist))

with open('nlab-terms.txt', 'a') as writer:
    for term in termlist:
        writer.write(term + '\n')


In [23]:
with open('tac-term-mappings.txt','a') as mappings:
    listy = []
    for line in open('tac-terms.txt','r').readlines():
        line = line.replace(' - ', '-')
        line = line.replace('$', '')
        line = line.replace(' ', '_')
        line = line[0].upper() + line[1:-1]
        if map.title_to_id(line):
            listy.append(line + ': ' + map.title_to_id(line)  + '\n')

    listy= sorted(set(listy))
    for line in listy:
        mappings.write(line)        
            


In [13]:
# get MISSING terms from wikidata
#term-id order, for tac stuff

map = WikiMapper('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/wikidata/index_enwiki-20190420.db')

wiki = open('france-missing.txt', 'a')
#wiki.write('ADJ-NOUN PAIRS:\n')
for line in open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/france/france.txt', 'r').readlines():
    line2 = line.split(', ')[1].strip()
    line2 = line2.replace(' - ', '-')
    line2 = line2.replace('$', '')
    line2 = line2.replace(' ', '_')
    line2 = line2[0].upper() + line2[1:]
    if not map.title_to_id(line2):
        wiki.write('- ' + line2 + '\n')

for line in open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/france/france-non-lean.txt','r').readlines():
    line = line.strip()
    line2 = line.replace(' - ', '-')
    line2 = line2.replace('$', '')
    line2 = line2.replace(' ', '_')
    line2 = line2[0].upper() + line2[1:]
    if not map.title_to_id(line2):
        wiki.write('- '+ line2 + '\n')


In [8]:
#id-term order, for tac stuff
map = WikiMapper('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/index_enwiki-20190420.db')

dictionary = dict()
wiki = open('tac-wikidata.txt', 'a')
#wiki.write('ADJ-NOUN PAIRS:\n')
for line in open('hf-ans.txt', 'r').readlines():
    line = line.replace(' - ', '-')
    line = line.replace('$', '')
    line = line.replace(' ', '_')
    line = line[0].upper() + line[1:line.index(':')]
    if map.title_to_id(line):
        dictionary[line] = map.title_to_id(line) 
        #wiki.write(map.title_to_id(line) + ': ' + line  + '\n')

#wiki.write('\nCOMPOUNDS:\n')
for line in open('hf-compounds.txt', 'r').readlines():
    line = line.replace(' - ', '-')
    line = line.replace('$', '')
    line = line.replace(' ', '_')
    line = line[0].upper() + line[1:line.index(':')]
    if map.title_to_id(line):
        dictionary[line] = map.title_to_id(line) 
        #wiki.write(map.title_to_id(line) + ': ' + line  + '\n')

#wiki.write('\nPROPER NOUNS:\n')
#for line in open('hf-propns.txt', 'r').readlines():
    #line = line.replace('$', '')
    #line = line.replace(' ', '_')
    #line = line[0].upper() + line[1:line.index(':')]
    #if map.title_to_id(line):
        #dictionary[line] = map.title_to_id(line) 
        #wiki.write(map.title_to_id(line) + ': ' + line  + '\n')

#wiki.write('\nNOUNS:\n')
for line in open('hf-nouns.txt', 'r').readlines():
    line = line.replace(' - ', '-')
    line = line.replace('$', '')
    line = line.replace(' ', '_')
    line = line[0].upper() + line[1:line.index(':')]
    if map.title_to_id(line):
        dictionary[line] = map.title_to_id(line) 
        #wiki.write(map.title_to_id(line) + ': ' + line  + '\n')

dictionary2 = sorted(dictionary)
print(dictionary2)

OperationalError: no such table: mapping

In [30]:
#this needs to be run twice to get past W for some reason, then just delete the repeated things.

map = WikiMapper('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/index_enwiki-20190420.db')

wiki = open('planetmath-wikidata.txt', 'a')
toread = open('planetmath.txt', 'r').readlines()
#toread = ['Cylindrical Coordinates\n', 'Cylindrical Coordinate System\n']
towrite = []
for line in toread:
    line = line[:-1]
    if map.title_to_id(line):
        towrite.append(line  +': ' +map.title_to_id(line) + '\n')
    else:
        line = line.replace(' - ', '-')
        line = line.replace('$', '')
        line = line.replace(' ', '_')
        if map.title_to_id(line):
            towrite.append(line  +': ' +map.title_to_id(line) + '\n')
        else:
            line = line.lower()
            line = line[0].upper() + line[1:]
            if map.title_to_id(line):
                towrite.append(line  +': ' +map.title_to_id(line) + '\n')

to = sorted(towrite)
for thing in to:
    wiki.write(thing)


In [28]:
planet = open('planetmath.txt','r').readlines()
moon = []
for line in planet:
    for term in line.split():
        moon.append(term)

moon2 = []
for term in moon:
    if term[0].isnumeric(): #remove leading numbers
        i = 0
        while term[i].isnumeric():
            term = term[i+1:]
    i = 0
    for letter in term:
        if i > 0 and term[i].isupper():
            term = term[:i] + '_' + term[i:]
            i = i + 1
        i = i + 1
    moon2.append(term)

moon2 = sorted(moon2)
with open('planetmath-clean.txt', 'a') as mon:
    for term in moon2:
        mon.write(term + '\n')

In [47]:
#for france
map = WikiMapper('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/wikidata/index_enwiki-20190420.db')
france = open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/france/france.txt','r').readlines()
towrite = []
for line in france:
    twos = line.split(', ')
    twos[1] = twos[1].replace(' - ', '-')
    twos[1] = twos[1][0].upper() + twos[1][1:].strip()
    twos[1] =  twos[1].replace('$', '')
    twos[1]= twos[1].replace(' ', '_')
    if twos[1][-1] == 's':
        twos[1] = twos[1][:-1]
    if map.title_to_id(twos[1]):
        towrite.append(map.title_to_id(twos[1]) + ': ' + twos[1] + '\n')
wiki = open('france-mappings.txt', 'a')
for thing in towrite:
    wiki.write(thing)


one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one
one


In [23]:
all = open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/chicago defs/defs.txt','r').readlines()
all = [term.strip() for term in all]

mappings = open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/chicago defs/manual-def-mappings.txt','r').readlines()

terms = []
for term in mappings:
    data = term.split(': ')
    terms.append(data[1].strip())


with open('chicago-missing.txt','a') as missing:
    for term in all:
        if not term in terms:
            missing.write("- " + term + '\n')

# NLAB

In [19]:
map = WikiMapper('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/wikidata/index_enwiki-20190420.db')

wikicats = ['_(mathematics)', '_(linear_algebra)', '_(algebraic_geometry)', '_(algebraic_topology)', '_(calculus)', '_(category_theory)',
             '_(commutative_algebra)', '_(field_theory)', '_(game_theory)', '_(topology)', '_(differential_geometry)', '_(graph_theory)', 
             '_(group_theory)', '_(invariant_theory)', '_(module_theory)', '_(order_theory)', '_(probability)', '_(ring_theory)',
             '_(representation_theory)', '_(set_theory)', '_(string_theory)', '_(symplectic geometry)', '_(tensor_theory)']

wiki = open('nlab-maps2.txt', 'a')

people = [name.strip() for name in open("/Users/lucyhorowitz/Documents/spaCy-Obsidian project/nlab/nlab-people.txt",'r').readlines()]
names = []
for person in people:
    for name in person.split():
        names.append(name)
        names.append(name + "'s")
for line in open("/Users/lucyhorowitz/Documents/spaCy-Obsidian project/nlab/nlab_page_names_raw.txt", "r").readlines():
    line = line.strip()
    if not line in people:
        line = line.replace(' - ', '-')
        line = line.replace('$', '')
        line = line.replace(' ', '_')
        line = line[0].upper() + line[1:]
        linesplus = [line + cat for cat in wikicats]
        exclude = False
        for thing in linesplus:
            if map.title_to_id(thing):
                exclude = True
        if not exclude:
            linesplus.append(line)
        for thing in linesplus:
            if map.title_to_id(thing):
                url = thing.replace('_', '+')
                url = url.replace("'", '%27')
                words = url.split('+')
                lower_first_letter = True
                if words[0] in names:
                     lower_first_letter = False
                if words[0].split('-')[0] in names:
                    lower_first_letter = False
                if words[0].split("%27")[0] in names:
                    lower_first_letter = False
                if words[0].isupper():
                    lower_first_letter = False
                if lower_first_letter:
                    url = url[0].lower() + url[1:]
                if "(" in url:
                    url = url[:url.index("(") - 1]
                wiki.write(thing.replace('_', " ") + "](https://ncatlab.org/nlab/show/" + url + '): ' + map.title_to_id(thing)  + '\n')

# HOSGOOD

In [13]:
import json

hosgood = json.load(open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/wikidata/hosgood.json'))
hmaps = open('hosgood-maps.txt', 'a')
for term in hosgood:
    try:
        hmaps.write(hosgood[term]["EN"]["atom"]+": "+hosgood[term]["refs"]["wikidata"] + "\n")
    except:
        print(hosgood[term]["EN"])

{'atom': 'locally Noetherian scheme', 'gend': ''}
{'atom': 'reduced scheme', 'gend': ''}
{'atom': 'integral scheme', 'gend': ''}
{'atom': 'separated scheme', 'gend': ''}
{'atom': 'irreducible scheme', 'gend': ''}
{'atom': 'quasi-compact scheme', 'gend': ''}
{'atom': 'scheme of finite type', 'gend': ''}
{'atom': 'reduced group', 'gend': ''}
{'atom': 'finitely presentable category', 'gend': ''}
{'atom': 'locally finitely presentable category', 'gend': ''}
{'atom': 'projective set', 'gend': ''}
{'atom': 'cofinal set', 'gend': ''}
{'atom': 'complementary set', 'gend': ''}
{'atom': 'smooth manifold', 'gend': ''}
{'atom': 'compact manifold', 'gend': ''}
{'atom': 'connected manifold', 'gend': ''}
{'atom': 'affix', 'gend': ''}
{'atom': 'trihedral angle', 'gend': ''}
{'atom': 'real axis', 'gend': ''}
{'atom': 'imaginary axis', 'gend': ''}


DENNIS

In [3]:
map = WikiMapper('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/wikidata/index_enwiki-20190420.db')

wikicats = ['_(mathematics)', '_(linear_algebra)', '_(algebraic_geometry)', '_(algebraic_topology)', '_(calculus)', '_(category_theory)',
             '_(commutative_algebra)', '_(field_theory)', '_(game_theory)', '_(topology)', '_(differential_geometry)', '_(graph_theory)', 
             '_(group_theory)', '_(invariant_theory)', '_(module_theory)', '_(order_theory)', '_(probability)', '_(ring_theory)',
             '_(representation_theory)', '_(set_theory)', '_(string_theory)', '_(symplectic geometry)', '_(tensor_theory)']

wiki = open('mathwords-mappings.txt', 'a')
#wiki.write('\nIN LEAN:\n')
for text in open('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/wikidata/mathwords.txt', 'r').readlines():
    line = text.strip()
    
    line = line.replace(' ', "_")
    
    line = line[0].upper() + line[1:]
    #print(line)
    linesplus = [line + cat for cat in wikicats]
    exclude = False
    for thing in linesplus:
        if map.title_to_id(thing):
            exclude = True
    if not exclude:
        linesplus.append(line)
    for thing in linesplus:
        if map.title_to_id(thing):
            wiki.write(thing.replace('_', " ") + ': ' + map.title_to_id(thing)  + '\n')


In [3]:
map = WikiMapper('/Users/lucyhorowitz/Documents/spaCy-Obsidian project/wikidata/index_enwiki-20190420.db')
map.title_to_id("Group")

'Q654302'